In [1]:
from dotenv import load_dotenv
import os

import geopandas as gpd
import pandas as pd
import math

from IPython.display import display, HTML
from sqlalchemy import create_engine

In [2]:
load_dotenv()

True

In [3]:
unit_link_gdf = gpd.read_file('./data/pedestrian_unit_link.gpkg')
unit_link_gdf.head()

,link_id,link_len,unit_link_id,unit_link_length,geometry
0,26893.0,16.464,1,16.46,"LINESTRING (126.98748 37.57053, 126.98747 37.5..."
1,65428.0,51.509,2,51.51,"LINESTRING (126.96522 37.59342, 126.96474 37.5..."
2,65440.0,100.509,3,19.58,"LINESTRING (126.9688 37.61426, 126.96877 37.61..."
3,65440.0,100.509,4,12.06,"LINESTRING (126.96877 37.61409, 126.96878 37.6..."
4,65440.0,100.509,5,13.39,"LINESTRING (126.96878 37.61398, 126.96883 37.6..."


In [4]:
unit_node_dem_gdf = gpd.read_file('./data/pedestrian_unit_node_dem.gpkg')
unit_node_dem_gdf.head()

,link_id,unit_link_id,vertex_index,vertex_part,vertex_part_index,distance,angle,elevation,geometry
0,26893.0,1,0,0,0,0.000000,357.706437,25.567894,POINT (126.98748 37.57053)
1,26893.0,1,1,0,1,0.000148,357.706437,25.696207,POINT (126.98747 37.57068)
2,65428.0,2,0,0,0,0.000000,297.492709,109.810547,POINT (126.96522 37.59342)
3,65428.0,2,1,0,1,0.000550,297.492709,107.327919,POINT (126.96474 37.59367)
4,65440.0,3,0,0,0,0.000000,191.498403,206.080276,POINT (126.9688 37.61426)


In [5]:
merged_gdf = unit_link_gdf.merge(unit_node_dem_gdf, on=['unit_link_id', 'unit_link_id'])
merged_gdf.head(10)

,link_id_x,link_len,unit_link_id,unit_link_length,geometry_x,link_id_y,vertex_index,vertex_part,vertex_part_index,distance,angle,elevation,geometry_y
0,26893.0,16.464,1,16.46,"LINESTRING (126.98748 37.57053, 126.98747 37.5...",26893.0,0,0,0,0.000000,357.706437,25.567894,POINT (126.98748 37.57053)
1,26893.0,16.464,1,16.46,"LINESTRING (126.98748 37.57053, 126.98747 37.5...",26893.0,1,0,1,0.000148,357.706437,25.696207,POINT (126.98747 37.57068)
2,65428.0,51.509,2,51.51,"LINESTRING (126.96522 37.59342, 126.96474 37.5...",65428.0,0,0,0,0.000000,297.492709,109.810547,POINT (126.96522 37.59342)
3,65428.0,51.509,2,51.51,"LINESTRING (126.96522 37.59342, 126.96474 37.5...",65428.0,1,0,1,0.000550,297.492709,107.327919,POINT (126.96474 37.59367)
4,65440.0,100.509,3,19.58,"LINESTRING (126.9688 37.61426, 126.96877 37.61...",65440.0,0,0,0,0.000000,191.498403,206.080276,POINT (126.9688 37.61426)
5,65440.0,100.509,3,19.58,"LINESTRING (126.9688 37.61426, 126.96877 37.61...",65440.0,1,0,1,0.000178,191.498403,208.772064,POINT (126.96877 37.61409)
6,65440.0,100.509,4,12.06,"LINESTRING (126.96877 37.61409, 126.96878 37.6...",65440.0,0,0,0,0.000000,173.737803,208.772064,POINT (126.96877 37.61409)
7,65440.0,100.509,4,12.06,"LINESTRING (126.96877 37.61409, 126.96878 37.6...",65440.0,1,0,1,0.000109,173.737803,211.487015,POINT (126.96878 37.61398)
8,65440.0,100.509,5,13.39,"LINESTRING (126.96878 37.61398, 126.96883 37.6...",65440.0,0,0,0,0.000000,154.738156,211.487015,POINT (126.96878 37.61398)
9,65440.0,100.509,5,13.39,"LINESTRING (126.96878 37.61398, 126.96883 37.6...",65440.0,1,0,1,0.000125,154.738156,212.492691,POINT (126.96883 37.61386)


In [6]:
# def calculate_slope(group):
#     group = group.sort_values(by='vertex_index')
#     elevation_change = group['elevation'].iloc[1] - group['elevation'].iloc[0]
#     length = group['length_x'].iloc[0]

#     if elevation_change == 0:
#         slope = 0
#     else:
#         slope = (elevation_change / length)

#     return slope

# merged_gdf['slope'] = merged_gdf.groupby('unit_link_id').apply(calculate_slope).reset_index(level=0, drop=True)

ulid_by_lid = {}
slope_by_ulid = {}

groups_by_ulid = merged_gdf.groupby('unit_link_id')

for (unit_link_id), groups in groups_by_ulid:
    # print("unit_link_id :: ", unit_link_id)
    # display(groups)
    
    elevation_change = abs(groups['elevation'].iloc[1] - groups['elevation'].iloc[0])
    length = groups['unit_link_length'].iloc[0]
    
    slope_ratio = 0 if elevation_change == 0 else elevation_change / length
    slope_percentage = slope_ratio * 100
    slope_degrees = math.degrees(math.atan(slope_ratio))

    ## 경사도 사전 세팅
    slope_by_ulid[unit_link_id] = slope_degrees
    
    ## lid-ulid 사전 세팅
    link_id = groups['link_id_x'].iloc[0]
    ulid_by_lid.setdefault(link_id, []).append(unit_link_id)


In [7]:
display( list(slope_by_ulid.items())[0:10] )
display( list(ulid_by_lid.items())[0:10] )

[(1, 0.44663718964306515),
 (2, 2.759349838547212),
 (3, 7.827748863532266),
 (4, 12.686942717273194),
 (5, 4.295222147286483),
 (6, 2.5269950193020945),
 (7, 3.355684263847089),
 (8, 3.9142614901282298),
 (9, 0.13251074620571873),
 (10, 0.5973625064279797)]

[(np.float64(26893.0), [1]),
 (np.float64(65428.0), [2]),
 (np.float64(65440.0), [3, 4, 5, 6, 7, 8]),
 (np.float64(67227.0), [9]),
 (np.float64(66088.0), [10]),
 (np.float64(66688.0), [11]),
 (np.float64(64891.0), [12, 13, 14, 15, 16]),
 (np.float64(64897.0), [17]),
 (np.float64(53993.0), [18]),
 (np.float64(73021.0), [19])]

In [8]:
merged_gdf.head()

## unit_link_id 별로 경사도 계산 (양쪽 노드 elevation으로 계산)
ulid_slope_df = merged_gdf.groupby('unit_link_id')['link_id_x'].min().reset_index()
ulid_slope_df = ulid_slope_df.rename(columns={'link_id_x': 'link_id'})
ulid_slope_df['ulid_slope'] = ulid_slope_df.apply(lambda r: slope_by_ulid[r['unit_link_id']] or 0, axis=1)
display( ulid_slope_df.head() )
# display( ulid_slope_df.info() )

## link_id 별로 경사도 통계
lid_stat_df = ulid_slope_df.groupby('link_id')['ulid_slope'].agg(['min', 'max', 'median', 'sum', 'count']).reset_index()
lid_stat_df['avg'] = lid_stat_df.apply(lambda r: r['sum'] / r['count'] if r['count'] > 0 else 0, axis=1)
lid_stat_df = lid_stat_df.rename(columns={
    'min': 'slope_min',
    'max': 'slope_max',
    'median': 'slope_median',
    'sum': 'slope_sum',
    'count': 'slope_count',
    'avg': 'slope_avg',
})
display( lid_stat_df.head() )
# display( lid_stat_df.info() )

,unit_link_id,link_id,ulid_slope
0,1,26893.0,0.446637
1,2,65428.0,2.759350
2,3,65440.0,7.827749
3,4,65440.0,12.686943
4,5,65440.0,4.295222


,link_id,slope_min,slope_max,slope_median,slope_sum,slope_count,slope_avg
0,1.0,0.381518,0.381518,0.381518,0.381518,1,0.381518
1,2.0,3.590154,12.633279,9.717532,35.658497,4,8.914624
2,3.0,2.913476,2.913476,2.913476,2.913476,1,2.913476
3,4.0,1.700435,1.700435,1.700435,1.700435,1,1.700435
4,5.0,8.631386,11.813902,10.222644,20.445288,2,10.222644


In [9]:
## 충정로역 온누리빌 7~9도 (스마트서울맵 비교)
display(ulid_slope_df.query('unit_link_id == 230547'))
display(ulid_slope_df.query('unit_link_id == 228424'))

display(lid_stat_df.query('link_id == 230823'))
display(lid_stat_df.query('link_id == 185499'))

,unit_link_id,link_id,ulid_slope
230546,230547,230823.0,8.425351


,unit_link_id,link_id,ulid_slope
228423,228424,185499.0,6.894111


,link_id,slope_min,slope_max,slope_median,slope_sum,slope_count,slope_avg
228163,230823.0,8.425351,8.425351,8.425351,8.425351,1,8.425351


,link_id,slope_min,slope_max,slope_median,slope_sum,slope_count,slope_avg
183372,185499.0,6.894111,6.894111,6.894111,6.894111,1,6.894111


In [10]:
## 신논현역 언덕 내리막길
display(ulid_slope_df.query('unit_link_id == 363830'))
display(ulid_slope_df.query('unit_link_id == 366535'))

display(lid_stat_df.query('link_id == 57714'))
display(lid_stat_df.query('link_id == 86230'))

,unit_link_id,link_id,ulid_slope
363829,363830,57714.0,3.645574


,unit_link_id,link_id,ulid_slope
366534,366535,86230.0,0.926331


,link_id,slope_min,slope_max,slope_median,slope_sum,slope_count,slope_avg
57056,57714.0,3.645574,3.645574,3.645574,3.645574,1,3.645574


,link_id,slope_min,slope_max,slope_median,slope_sum,slope_count,slope_avg
85226,86230.0,0.926331,0.926331,0.926331,0.926331,1,0.926331


In [11]:
display(ulid_slope_df.describe())
display(lid_stat_df.describe())

lid_stat_df[
    lid_stat_df['slope_count'] < 2
]

,unit_link_id,link_id,ulid_slope
count,419837.00000,419837.000000,419837.000000
mean,209919.00000,141910.099177,2.391581
std,121196.64682,81729.892337,3.721775
min,1.00000,1.000000,0.000000
25%,104960.00000,70973.000000,0.297398
50%,209919.00000,143775.000000,0.958045
75%,314878.00000,212549.000000,2.888170
max,419837.00000,282285.000000,72.947219


,link_id,slope_min,slope_max,slope_median,slope_sum,slope_count,slope_avg
count,279016.000000,279016.000000,279016.000000,279016.000000,279016.000000,279016.000000,279016.000000
mean,141133.431810,1.734567,2.457006,2.035332,3.598626,1.504706,2.065040
std,81486.237946,2.794432,3.879595,3.063748,8.861423,1.316561,3.071542
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,70565.750000,0.215421,0.338251,0.299002,0.358323,1.000000,0.308831
50%,141114.500000,0.691376,1.012512,0.873219,1.123422,1.000000,0.897716
75%,211694.250000,2.021097,2.879747,2.444915,3.380844,2.000000,2.503466
max,282285.000000,58.355503,72.947219,58.355503,909.046754,91.000000,58.355503


,link_id,slope_min,slope_max,slope_median,slope_sum,slope_count,slope_avg
0,1.0,0.381518,0.381518,0.381518,0.381518,1,0.381518
2,3.0,2.913476,2.913476,2.913476,2.913476,1,2.913476
3,4.0,1.700435,1.700435,1.700435,1.700435,1,1.700435
6,7.0,0.096417,0.096417,0.096417,0.096417,1,0.096417
8,9.0,0.604496,0.604496,0.604496,0.604496,1,0.604496
...,...,...,...,...,...,...,...
279011,282280.0,2.491970,2.491970,2.491970,2.491970,1,2.491970
279012,282282.0,0.279293,0.279293,0.279293,0.279293,1,0.279293
279013,282283.0,0.609315,0.609315,0.609315,0.609315,1,0.609315
279014,282284.0,4.841235,4.841235,4.841235,4.841235,1,4.841235


In [12]:
count_summary = lid_stat_df['slope_count'].value_counts().reset_index()
count_summary.columns = ['slope_count', 'frequency']
count_summary = count_summary.sort_values(by='slope_count', ascending=True)
display(count_summary.head(10))

,slope_count,frequency
0,1,208604
1,2,39546
2,3,15609
3,4,6865
4,5,3454
5,6,1888
6,7,1046
7,8,665
8,9,406
9,10,264


In [13]:
# ## load to postgis 
# db_user = os.getenv('DB_USER')
# db_password = os.getenv('DB_PASSWORD')
# db_host = os.getenv('DB_HOST')
# db_port = os.getenv('DB_PORT')
# db_name = os.getenv('DB_NAME')

# engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
# print(engine.name, engine.url)

# table_name = 'pedestrian_link_slope_stat'
# slope_stats_gdf.to_sql(table_name, engine, if_exists='append', chunksize=1000, index=False)